In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import cProfile

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import xgi

from governance import *

In [ ]:
num_decisions = 6
num_people = 10000

select_decision_type = "snowball"
select_group_type = "star"
make_decision_type = "average"
update_opinions_type = "star"

# opinions = create_opinions(num_people, num_decisions)
opinions = create_polarized_opinions(
    num_people, num_decisions, [1, -1, -1, -1, -1, 1], 0.2, 0.2
)
votes = 2 * (np.mean(opinions, axis=0) > 0) - 1

# decision_matrix = create_decision_matrix(num_decisions)
decision_matrix = np.array(
    [
        [0, -1, -1, -1, -1, 1],
        [-1, 0, 0, 0, 0, 0],
        [-1, 0, 0, 0, 0, 0],
        [-1, 0, 0, 0, 0, 0],
        [-1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
    ]
)
print(decision_matrix)

In [ ]:
group_size = 51
overlap = 6

# opinions = create_opinions(num_people, num_decisions)
opinions = create_polarized_opinions(
    num_people, num_decisions, [1, -1, -1, -1, -1, 1], 0.33, 0.6
)
cProfile.run(
    "decision_process(opinions, decision_matrix, group_size, overlap, select_decision_type=select_decision_type, select_group_type=select_group_type, make_decision_type=make_decision_type, update_opinions_type=update_opinions_type)"
)

In [ ]:
group_size = 51
overlap = 6

# opinions = create_opinions(num_people, num_decisions)
opinions = create_polarized_opinions(
    num_people, num_decisions, [1, -1, -1, -1, -1, 1], 0.33, 0.6
)
decisions, final_opinions, decision_groups = decision_process(
    opinions,
    decision_matrix,
    group_size,
    overlap,
    select_decision_type=select_decision_type,
    select_group_type=select_group_type,
    make_decision_type=make_decision_type,
    update_opinions_type=update_opinions_type,
)
# d = decisions_to_array(decisions)
H = decision_groups
coherence = 0
for d1, d2 in np.ndindex(decision_matrix.shape):
    coherence += decision_matrix[d1, d2] * decisions[d1] * decisions[d2]

fig = matplotlib.pyplot.figure()
xgi.draw(H, xgi.pairwise_spring_layout(H), node_size=7, hull=True, radius=0.05)

fig.savefig("Figures/example_hypergraph.pdf", dpi=1000)
fig.savefig("Figures/example_hypergraph.png", dpi=1000)

In [ ]:
method = "polarized"

In [ ]:
num_samples = 100
group_sizes = np.arange(1, 51, 2, dtype=int)
overlap_sizes = np.arange(0, 21, 1, dtype=int)
satisfaction_group_size = np.zeros((len(group_sizes), num_samples))
satisfiability_group_size = np.zeros((len(group_sizes), num_samples))
overlap = 0
H_group_sizes = dict()

for i in range(len(group_sizes)):
    g = group_sizes[i]
    print(i / len(group_sizes))
    for j in range(num_samples):
        if method == "random":
            opinions = create_random_opinions(num_people, num_decisions)
        elif method == "polarized":
            opinions = create_polarized_opinions(
                num_people, num_decisions, [1, -1, -1, -1, -1, 1], 0.33, 0.6
            )
        elif method == "zealots":
            opinions = create_mixed_opinions(
                num_people, num_decisions, [0.6, -0.4, -0.4, -0.4, -0.4, 0.6], 0.0, 0.7
            )
        elif method == "incoherent":
            opinions = create_incoherent_opinions(
                num_people, num_decisions, [0.6, 0.4, 0.4, 0.4, 0.4, -0.6]
            )
        # binary_votes = 2*(np.mean(opinions, axis=0)>0)-1
        votes = np.mean(opinions, axis=0)
        decisions, final_opinions, decision_groups = decision_process(
            opinions,
            decision_matrix,
            g,
            overlap,
            select_decision_type=select_decision_type,
            select_group_type=select_group_type,
            make_decision_type=make_decision_type,
            update_opinions_type=update_opinions_type,
        )
        # H_group_sizes[(g, overlap, j)] = decision_groups
        d = decisions_to_array(decisions)
        # satisfaction_group_size[i] += np.mean(opinions.dot(d))/num_samples
        satisfaction_group_size[i, j] += votes.dot(d) / np.sum(
            np.abs(votes)
        )  # / num_samples
        for d1, d2 in np.ndindex(decision_matrix.shape):
            satisfiability_group_size[i, j] += (
                decision_matrix[d1, d2] * decisions[d1] * decisions[d2]  # / num_samples
            )

In [ ]:
num_samples = 100
group_sizes = np.arange(1, 51, 2, dtype=int)
overlap_sizes = np.arange(0, 21, 1, dtype=int)
satisfaction_overlap_size = np.zeros((len(overlap_sizes), num_samples))
satisfiability_overlap_size = np.zeros((len(overlap_sizes), num_samples))
group_size = 50  # max(overlap_sizes) + 1

H_overlap_sizes = dict()

for i in range(len(overlap_sizes)):
    o = overlap_sizes[i]
    print(i / len(overlap_sizes))
    for j in range(num_samples):
        if method == "random":
            opinions = create_random_opinions(num_people, num_decisions)
        elif method == "polarized":
            opinions = create_polarized_opinions(
                num_people, num_decisions, [1, -1, -1, -1, -1, 1], 0.33, 0.6
            )
        elif method == "zealots":
            opinions = create_mixed_opinions(
                num_people, num_decisions, [0.6, -0.4, -0.4, -0.4, -0.4, 0.6], 0.0, 0.7
            )
        elif method == "incoherent":
            opinions = create_incoherent_opinions(
                num_people, num_decisions, [0.6, 0.4, 0.4, 0.4, 0.4, -0.6]
            )
        # binary_votes = 2*(np.mean(opinions, axis=0)>0)-1
        votes = np.mean(opinions, axis=0)
        decisions, final_opinions, decision_groups = decision_process(
            opinions,
            decision_matrix,
            group_size,
            o,
            select_decision_type=select_decision_type,
            select_group_type=select_group_type,
            make_decision_type=make_decision_type,
            update_opinions_type=update_opinions_type,
        )
        H_overlap_sizes[(group_size, o, j)] = decision_groups
        d = decisions_to_array(decisions)
        # satisfaction_overlap_size[i] += np.mean(opinions.dot(d))/num_samples
        satisfaction_overlap_size[i, j] += votes.dot(d) / np.sum(
            np.abs(votes)
        )  # / num_samples
        for d1, d2 in np.ndindex(decision_matrix.shape):
            satisfiability_overlap_size[i, j] += (
                decision_matrix[d1, d2] * decisions[d1] * decisions[d2]  # / num_samples
            )

In [ ]:
plt.figure()
plt.subplot(211)
plt.errorbar(
    group_sizes,
    satisfaction_group_size.mean(axis=1),
    yerr=satisfaction_group_size.std(axis=1),
    color="black",
    marker="o",
)
plt.ylabel("Average satisfaction")
plt.xlabel("Group size")
sns.despine()

plt.subplot(212)
plt.errorbar(
    overlap_sizes,
    satisfaction_overlap_size.mean(axis=1),
    yerr=satisfaction_overlap_size.std(axis=1),
    color="black",
    marker="o",
)
plt.ylabel("Average satisfaction")
plt.xlabel("Overlap size")
plt.tight_layout()
sns.despine()


plt.show()

In [ ]:
plt.figure()
plt.subplot(211)
plt.errorbar(
    group_sizes,
    satisfiability_group_size.mean(axis=1),
    yerr=satisfiability_group_size.std(axis=1),
    color="black",
    marker="o",
)
plt.ylabel("Average coherence")
plt.xlabel("Group size")
plt.subplot(212)
plt.errorbar(
    overlap_sizes,
    satisfiability_overlap_size.mean(axis=1),
    yerr=satisfiability_overlap_size.std(axis=1),
    color="black",
    marker="o",
)
plt.ylabel("Average coherence")
plt.xlabel("Overlap size")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.subplot(221)
plt.errorbar(
    group_sizes,
    satisfaction_group_size.mean(axis=1),
    yerr=satisfaction_group_size.std(axis=1),
    color="black",
    marker="o",
)
plt.ylabel("Average satisfaction")
plt.xlabel("Group size")
sns.despine()

plt.subplot(222)
plt.errorbar(
    overlap_sizes,
    satisfaction_overlap_size.mean(axis=1),
    yerr=satisfaction_overlap_size.std(axis=1),
    color="black",
    marker="o",
)
plt.ylabel("Average satisfaction")
plt.xlabel("Overlap size")
sns.despine()

plt.subplot(223)
plt.errorbar(
    group_sizes,
    satisfiability_group_size.mean(axis=1) / 10,
    yerr=satisfiability_group_size.std(axis=1) / 10,
    color="black",
    marker="o",
)
plt.ylabel("Average coherence")
plt.xlabel("Group size")
sns.despine()

plt.subplot(224)
plt.errorbar(
    overlap_sizes,
    satisfiability_overlap_size.mean(axis=1) / 10,
    yerr=satisfiability_overlap_size.std(axis=1) / 10,
    color="black",
    marker="o",
)
plt.ylabel("Average coherence")
plt.xlabel("Overlap size")
sns.despine()

plt.tight_layout()
plt.show()

In [ ]:
num_samples = 100
group_sizes = np.arange(1, 51, 2, dtype=int)
overlap_sizes = np.arange(0, 21, 1, dtype=int)
satisfaction_heatmap = np.zeros((len(group_sizes), len(overlap_sizes)))
satisfiability_heatmap = np.zeros((len(group_sizes), len(overlap_sizes)))
H_group_sizes = dict()

for i in range(len(group_sizes)):
    g = group_sizes[i]
    for j in range(len(overlap_sizes)):
        o = overlap_sizes[j]
        if o > g:
            satisfaction_heatmap[i, j] = np.nan
            satisfiability_heatmap[i, j] = np.nan
        else:
            print(i / len(group_sizes), o / len(overlap_sizes))
            for n in range(num_samples):
                if method == "random":
                    opinions = create_random_opinions(num_people, num_decisions)
                elif method == "polarized":
                    opinions = create_polarized_opinions(
                        num_people, num_decisions, [1, -1, -1, -1, -1, 1], 0.33, 0.6
                    )
                elif method == "zealots":
                    opinions = create_mixed_opinions(
                        num_people,
                        num_decisions,
                        [0.6, -0.4, -0.4, -0.4, -0.4, 0.6],
                        0.0,
                        0.7,
                    )
                elif method == "incoherent":
                    opinions = create_incoherent_opinions(
                        num_people, num_decisions, [0.6, 0.4, 0.4, 0.4, 0.4, -0.6]
                    )
                votes = np.mean(opinions, axis=0)
                decisions, final_opinions, decision_groups = decision_process(
                    opinions,
                    decision_matrix,
                    g,
                    o,
                    select_decision_type=select_decision_type,
                    select_group_type=select_group_type,
                    make_decision_type=make_decision_type,
                    update_opinions_type=update_opinions_type,
                )
                # H_group_sizes[(g, overlap, j)] = decision_groups
                d = decisions_to_array(decisions)
                # satisfaction_group_size[i] += np.mean(opinions.dot(d))/num_samples
                satisfaction_heatmap[i, j] += (
                    votes.dot(d) / np.sum(np.abs(votes))
                ) / num_samples
                for d1, d2 in np.ndindex(decision_matrix.shape):
                    satisfiability_heatmap[i, j] += (
                        decision_matrix[d1, d2]
                        * decisions[d1]
                        * decisions[d2]
                        / num_samples
                    )


if method == "random":
    np.savetxt(
        "Simulations/random_satisfaction_heatmap_sigma1.csv",
        satisfaction_heatmap,
        delimiter=",",
    )
    np.savetxt(
        "Simulations/random_satisfiability_heatmap_sigma1.csv",
        satisfiability_heatmap,
        delimiter=",",
    )
elif method == "polarized":
    np.savetxt(
        "Simulations/polarized_satisfaction_heatmap_sigma1.csv",
        satisfaction_heatmap,
        delimiter=",",
    )
    np.savetxt(
        "Simulations/polarized_satisfiability_heatmap_sigma1.csv",
        satisfiability_heatmap,
        delimiter=",",
    )
elif method == "zealots":
    np.savetxt(
        "Simulations/zealots_satisfaction_heatmap_sigma1.csv",
        satisfaction_heatmap,
        delimiter=",",
    )
    np.savetxt(
        "Simulations/zealots_satisfiability_heatmap_sigma1.csv",
        satisfiability_heatmap,
        delimiter=",",
    )
elif method == "incoherent":
    np.savetxt(
        "Simulations/incoherent_satisfaction_heatmap_sigma1.csv",
        satisfaction_heatmap,
        delimiter=",",
    )
    np.savetxt(
        "Simulations/incoherent_satisfiability_heatmap_sigma1.csv",
        satisfiability_heatmap,
        delimiter=",",
    )

In [ ]:
plt.figure()
plt.subplot(211)
hm1 = plt.imshow(satisfaction_heatmap)
plt.colorbar(hm1)
plt.ylabel("Group size")
plt.xlabel("Overlap size")
plt.subplot(212)
hm2 = plt.imshow(satisfiability_heatmap / 10)
plt.colorbar(hm2)
plt.ylabel("Group size")
plt.xlabel("Overlap size")
plt.tight_layout()
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

font = {"family": "normal", "weight": "normal", "size": 18}

matplotlib.rc("font", **font)
kwargs = {"linestyle": "--"}

fig, [[ax1, ax2], [ax3, ax4]] = plt.subplots(
    2, 2, figsize=(16, 12), gridspec_kw={"height_ratios": [2, 1]}
)

divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05)
hm1 = ax1.imshow(satisfaction_heatmap)  # , cmap='Greys')
cbar1 = plt.colorbar(hm1, cax=cax)
cbar1.set_label("Satisfaction")
ax1.set_ylabel("Group size")
ax1.set_xlabel("Overlap size")
ax1.set_yticks([0, 5, 10, 15, 20])
ax1.set_yticklabels([1, 11, 21, 31, 41])
ax1.set_xticks([0, 5, 10, 15, 20])


divider = make_axes_locatable(ax2)
cax = divider.append_axes("right", size="5%", pad=0.05)
hm2 = ax2.imshow(satisfiability_heatmap / 10)  # , cmap='Greys')
cbar2 = plt.colorbar(hm2, cax=cax)
cbar2.set_label("Coherence")
ax2.set_ylabel("Group size")
ax2.set_xlabel("Overlap size")
ax2.set_yticks([0, 5, 10, 15, 20])
ax2.set_yticklabels([1, 11, 21, 31, 41])
ax2.set_xticks([0, 5, 10, 15, 20])

ax3.errorbar(
    group_sizes,
    satisfaction_group_size.mean(axis=1),
    yerr=1.9723 * (satisfaction_group_size.std(axis=1)) / np.sqrt(num_samples),
    color="tab:blue",
    marker="o",
    ms=7,
    label="Satisfaction",
)
ax3.errorbar(
    group_sizes,
    satisfiability_group_size.mean(axis=1) / 10,
    yerr=(satisfiability_group_size.std(axis=1) / 10),
    color="tab:orange",
    marker="o",
    label="Coherence",
)
ax3.set_ylabel("Satisfaction/Coherence")
ax3.set_xlabel("Group size")
ax3.set_xticks([0, 10, 20, 30, 40, 50])
ax3.axhline(0, 0, 50, color="black", **kwargs)
ax3.legend(frameon=False, bbox_to_anchor=(0.9, 0.3))
ax3.text(0.5, 1.2, "Overlap = 0")
sns.despine()

ax4.errorbar(
    overlap_sizes,
    satisfaction_overlap_size.mean(axis=1),
    yerr=(satisfaction_overlap_size.std(axis=1)),
    color="tab:blue",
    marker="o",
    ms=7,
    label="Satisfaction",
)
ax4.errorbar(
    overlap_sizes,
    satisfiability_overlap_size.mean(axis=1) / 10,
    yerr=(satisfiability_overlap_size.std(axis=1) / 10),
    color="tab:orange",
    marker="o",
    label="Coherence",
)
ax4.set_ylabel("Satisfaction/Coherence")
ax4.set_xlabel("Overlap size")
ax4.set_xticks([0, 5, 10, 15, 20])
ax4.axhline(0, 0, 50, color="black", **kwargs)
ax4.text(0.5, 1.2, "Group size = 50")
sns.despine()

plt.tight_layout()


if method == "random":
    plt.savefig("Figures/complete_random_sigma1.pdf")
    plt.savefig("Figures/complete_random_sigma1.png", dpi=1000)
elif method == "polarized":
    plt.savefig("Figures/complete_polarized_sigma1.pdf")
    plt.savefig("Figures/complete_polarized_sigma1.png", dpi=1000)
elif method == "zealots":
    plt.savefig("Figures/complete_zealots_sigma1.pdf")
    plt.savefig("Figures/complete_zealots_sigma1.png", dpi=1000)
elif method == "incoherent":
    plt.savefig("Figures/complete_incoherent_sigma1.pdf")
    plt.savefig("Figures/complete_incoherent_sigma1.png", dpi=1000)
plt.show()

In [ ]:
key = (50, 20, 1)
xgi.draw(
    H_overlap_sizes[key],
    xgi.pairwise_spring_layout(H_overlap_sizes[key]),
    node_size=5,
    hull=True,
    radius=0.05,
)

In [ ]:
key = (20, 1, 1)
xgi.draw(
    H_group_sizes[key],
    xgi.pairwise_spring_layout(H_group_sizes[key]),
    node_size=5,
    hull=True,
)

## 